# BERT Fine-Tuning Tutorial with PyTorch

By Chris McCormick and Nick Ryan

*Revised on 3/20/20 - Switched to `tokenizer.encode_plus` and added validation loss. See [Revision History](https://colab.research.google.com/drive/1pTuQhug6Dhl9XalKB0zUGf4FIdYFlpcX#scrollTo=IKzLS9ohzGVu) at the end for details.*




In this tutorial I'll show you how to use BERT with the huggingface PyTorch library to quickly and efficiently fine-tune a model to get near state of the art performance in sentence classification. More broadly, I describe the practical application of transfer learning in NLP to create high performance models with minimal effort on a range of NLP tasks.

This post is presented in two forms--as a blog post [here](http://mccormickml.com/2019/07/22/BERT-fine-tuning/) and as a Colab Notebook [here](https://colab.research.google.com/drive/1pTuQhug6Dhl9XalKB0zUGf4FIdYFlpcX). 

The content is identical in both, but: 
* The blog post includes a comments section for discussion. 
* The Colab Notebook will allow you to run the code and inspect it as you read through.

I've also published a video walkthrough of this post on my YouTube channel! [Part 1](https://youtu.be/x66kkDnbzi4) and [Part 2](https://youtu.be/Hnvb9b7a_Ps).



# Contents

See "Table of contents" in the sidebar to the left.

# Introduction


## History

2018 was a breakthrough year in NLP. Transfer learning, particularly models like Allen AI's ELMO, OpenAI's Open-GPT, and Google's BERT allowed researchers to smash multiple benchmarks with minimal task-specific fine-tuning and provided the rest of the NLP community with pretrained models that could easily (with less data and less compute time) be fine-tuned and implemented to produce state of the art results. Unfortunately, for many starting out in NLP and even for some experienced practicioners, the theory and practical application of these powerful models is still not well understood.



## What is BERT?

BERT (Bidirectional Encoder Representations from Transformers), released in late 2018, is the model we will use in this tutorial to provide readers with a better understanding of and practical guidance for using transfer learning models in NLP. BERT is a method of pretraining language representations that was used to create models that NLP practicioners can then download and use for free. You can either use these models to extract high quality language features from your text data, or you can fine-tune these models on a specific task (classification, entity recognition, question answering, etc.) with your own data to produce state of the art predictions.

This post will explain how you can modify and fine-tune BERT to create a powerful NLP model that quickly gives you state of the art results. 



## Advantages of Fine-Tuning



In this tutorial, we will use BERT to train a text classifier. Specifically, we will take the pre-trained BERT model, add an untrained layer of neurons on the end, and train the new model for our classification task. Why do this rather than train a train a specific deep learning model (a CNN, BiLSTM, etc.) that is well suited for the specific NLP task you need? 

1. **Quicker Development**

    * First, the pre-trained BERT model weights already encode a lot of information about our language. As a result, it takes much less time to train our fine-tuned model - it is as if we have already trained the bottom layers of our network extensively and only need to gently tune them while using their output as features for our classification task. In fact, the authors recommend only 2-4 epochs of training for fine-tuning BERT on a specific NLP task (compared to the hundreds of GPU hours needed to train the original BERT model or a LSTM from scratch!). 

2. **Less Data**

    * In addition and perhaps just as important, because of the pre-trained weights this method allows us to fine-tune our task on a much smaller dataset than would be required in a model that is built from scratch. A major drawback of NLP models built from scratch is that we often need a prohibitively large dataset in order to train our network to reasonable accuracy, meaning a lot of time and energy had to be put into dataset creation. By fine-tuning BERT, we are now able to get away with training a model to good performance on a much smaller amount of training data.

3. **Better Results**

    * Finally, this simple fine-tuning procedure (typically adding one fully-connected layer on top of BERT and training for a few epochs) was shown to achieve state of the art results with minimal task-specific adjustments for a wide variety of tasks: classification, language inference, semantic similarity, question answering, etc. Rather than implementing custom and sometimes-obscure architetures shown to work well on a specific task, simply fine-tuning BERT is shown to be a better (or at least equal) alternative.



### A Shift in NLP

This shift to transfer learning parallels the same shift that took place in computer vision a few years ago. Creating a good deep learning network for computer vision tasks can take millions of parameters and be very expensive to train. Researchers discovered that deep networks learn hierarchical feature representations (simple features like edges at the lowest layers with gradually more complex features at higher layers). Rather than training a new network from scratch each time, the lower layers of a trained network with generalized image features could be copied and transfered for use in another network with a different task. It soon became common practice to download a pre-trained deep network and quickly retrain it for the new task or add additional layers on top - vastly preferable to the expensive process of training a network from scratch. For many, the introduction of deep pre-trained language models in 2018 (ELMO, BERT, ULMFIT, Open-GPT, etc.) signals the same shift to transfer learning in NLP that computer vision saw.

Let's get started!

# 1. Setup

## 1.1. Using Colab GPU for Training



Google Colab offers free GPUs and TPUs! Since we'll be training a large neural network it's best to take advantage of this (in this case we'll attach a GPU), otherwise training will take a very long time.

A GPU can be added by going to the menu and selecting:

`Edit 🡒 Notebook Settings 🡒 Hardware accelerator 🡒 (GPU)`

Then run the following cell to confirm that the GPU is detected.

In [0]:
!nvidia-smi

Fri May 15 23:01:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In order for torch to use the GPU, we need to identify and specify the GPU as the device. Later, in our training loop, we will load data onto the device. 

In [0]:
import torch
device = torch.device("cpu")

In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P40


In [3]:
torch.cuda.empty_cache()

## 1.2. Installing the Hugging Face Library



Next, let's install the [transformers](https://github.com/huggingface/transformers) package from Hugging Face which will give us a pytorch interface for working with BERT. (This library contains interfaces for other pretrained language models like OpenAI's GPT and GPT-2.) We've selected the pytorch interface because it strikes a nice balance between the high-level APIs (which are easy to use but don't provide insight into how things work) and tensorflow code (which contains lots of details but often sidetracks us into lessons about tensorflow, when the purpose here is BERT!).

At the moment, the Hugging Face library seems to be the most widely accepted and powerful pytorch interface for working with BERT. In addition to supporting a variety of different pre-trained transformer models, the library also includes pre-built modifications of these models suited to your specific task. For example, in this tutorial we will use `BertForSequenceClassification`.

The library also includes task-specific classes for token classification, question answering, next sentence prediciton, etc. Using these pre-built classes simplifies the process of modifying BERT for your purposes.


In [4]:
!pip install transformers

The code in this notebook is actually a simplified version of the [run_glue.py](https://github.com/huggingface/transformers/blob/master/examples/run_glue.py) example script from huggingface.

`run_glue.py` is a helpful utility which allows you to pick which GLUE benchmark task you want to run on, and which pre-trained model you want to use (you can see the list of possible models [here](https://github.com/huggingface/transformers/blob/e6cff60b4cbc1158fbd6e4a1c3afda8dc224f566/examples/run_glue.py#L69)). It also supports using either the CPU, a single GPU, or multiple GPUs. It even supports using 16-bit precision if you want further speed up.

Unfortunately, all of this configurability comes at the cost of *readability*. In this Notebook, we've simplified the code greatly and added plenty of comments to make it clear what's going on. 

# 2. Loading MultiFC Dataset


## 2.1. Download & Extract

We'll use the `wget` package to download the dataset to the Colab instance's file system. 

In [5]:
!pip install wget

In [6]:
import wget
import os

print('Downloading dataset...')

# The URL for the dataset zip file.
url = 'https://competitions.codalab.org/my/datasets/download/4db8bf21-def7-4a86-99f5-7b23d5691bb3'

# Download the file (if we haven't already)
if not os.path.exists('multi-fc/'):
    !mkdir multi-fc
    wget.download(url, 'multi-fc/multi-fc.zip')
    !unzip multi-fc/multi-fc.zip -d multi-fc/

## 2.2. Parse

We can see from the file names that both `tokenized` and `raw` versions of the data are available. 

We can't use the pre-tokenized version because, in order to apply the pre-trained BERT, we *must* use the tokenizer provided by the model. This is because (1) the model has a specific, fixed vocabulary and (2) the BERT tokenizer has a particular way of handling out-of-vocabulary words.

We'll use pandas to parse the "in-domain" training set and look at a few of its properties and data points.

In [7]:
import pandas as pd
import csv

# Load the dataset into a pandas dataframe.
train_df = pd.read_csv("multi-fc/train.tsv", delimiter='\t', header=None, quoting=csv.QUOTE_NONE, \
                 names= ['claimID', 'claim', 'label', 'claimURL', 'reason', 'categories', 'speaker', \
                  'checker', 'tags', 'articleTitle', 'publishDate', 'claimDate', 'entities'])

dev_df = pd.read_csv("multi-fc/dev.tsv", delimiter='\t', header=None, quoting=csv.QUOTE_NONE, \
                 names= ['claimID', 'claim', 'label', 'claimURL', 'reason', 'categories', 'speaker', \
                  'checker', 'tags', 'articleTitle', 'publishDate', 'claimDate', 'entities'])

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(train_df.shape[0]))

# Report the number of sentences.
print('Number of dev sentences: {:,}\n'.format(dev_df.shape[0]))

# Display 2 random rows from the data.
train_df.head()

Number of training sentences: 27,940

Number of dev sentences: 3,493



,claimID,claim,label,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities
0,pomt-03627,"""Six out of 10 of the highest unemployment rat...",half-true,/ohio/statements/2013/may/06/chris-redfern/ohi...,When a couple of Statehouse Republicans prepar...,None,Chris Redfern,None,None,None,2013-05-06T06:00:00,2013-04-30,['None']
1,pomt-09611,"""No Democratic campaign for (Fla.) governor ha...",true,/florida/statements/2010/jan/15/alex-sink/flor...,Florida's leading Republican candidate for gov...,None,Alex Sink,None,None,None,2010-01-15T13:59:00,2010-01-06,['None']
2,tron-00214,Forward an email for Jasmine,fiction!,https://www.truthorfiction.com/jasmine/,None,9-11-attack,None,None,None,Forward an email for Jasmine,"Mar 17, 2015",None,['None']
3,snes-04484,Pope Francis endorsed Donald Trump for president.,false,https://www.snopes.com/fact-check/pope-francis...,None,Junk News,None,Dan Evon,None,"Pope Francis Shocks World, Endorses Donald Tru...",10 July 2016,None,['None']
4,pomt-06704,Says Ron Paul insisted FEMA should be shut down.,true,/texas/statements/2011/sep/03/maureen-dowd/mau...,Commenting on the federal response to Hurrican...,None,Maureen Dowd,None,None,None,2011-09-03T06:00:00,2011-08-30,['None']


In [8]:
def generate_domain_df(df):
    getDomain = lambda x: x[:4]
    return df['claimID'].apply(getDomain).reset_index().rename(columns={"claimID": "domain"})

In [9]:
def sample_df(df, sample_frac=0.1):
    getDomain = lambda x: x[:4]
    df['domain'] = df['claimID'].apply(getDomain)
    df = df.groupby('domain').apply(lambda x: x.sample(frac = sample_frac))
    df.reset_index(drop=True,inplace=True)
    df.drop(columns=['domain'],inplace=True)
    return df

In [10]:
def drop_malformed(df):
    return df.dropna(subset=['claim']).reset_index()

(**A**) adding getSnippets() method 

In [11]:
def get_snippets(claimID):
  snippets = []
  try:
    f=open("multi-fc/snippets/{claimID}".format(claimID=claimID), "r")
    for line in f.readlines():
        split = line.split("\t")
        snippet = split[2]
        snippets.append(snippet)
    return snippets
  except FileNotFoundError:
    ##claimID did not have any snippets (we have 3875 / len(train) instances of this)
    return snippets


In [12]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_df['label'])

LabelEncoder()

In [13]:
# train_df = sample_df(train_df)

In [14]:
#gives original train and dev DFs with just the bad claims dropped
train_df = drop_malformed(train_df)
dev_df = drop_malformed(dev_df)

In [15]:
##gives df of domain for every claim
train_domains = generate_domain_df(train_df)
dev_domains = generate_domain_df(dev_df)

##maps domain name to id from 0 to domains - 1 
domain_to_domain_id = {k: v for v, k in enumerate(train_domains.domain.unique())}

## replaces domain name column with domain id
train_domains = train_domains.replace({'domain': domain_to_domain_id})
dev_domains = dev_domains.replace({'domain': domain_to_domain_id})

In [16]:
## creates dataframe with claim, index, domain id
train_claims = pd.concat([train_df[['claim']], train_domains], axis=1)
dev_claims = pd.concat([dev_df[['claim']], dev_domains], axis=1)


In [17]:
##gives dfs with label, index, domain_id
train_labels = pd.concat([train_df['label'], train_domains], axis=1)
dev_labels = pd.concat([dev_df['label'], dev_domains], axis=1)

##creates dictionary mapping domain_id to all possible labels within domain
train_labels_map_to_domain = train_labels.groupby('domain')['label'].apply(set).apply(list).to_dict()
dev_labels_map_to_domain = dev_labels.groupby('domain')['label'].apply(set).apply(list).to_dict()

##maps possible labels within to domain to 0 to num_labels-1
for domain in train_labels_map_to_domain:
    train_labels_map_to_domain[domain] = {k: v for v, k in enumerate(train_labels_map_to_domain[domain])} 



## creates list of size len(train) and len(dev) where each label is mapped to its' value within its' domain as per previous code
original_train_labels = train_labels
train_labels = []
original_dev_labels = dev_labels
dev_labels = []
for index, row in original_train_labels.iterrows():
    train_labels.append(train_labels_map_to_domain[row.domain][row.label])
for index, row in original_dev_labels.iterrows():
    dev_labels.append(train_labels_map_to_domain[row.domain][row.label])

In [18]:
## creates dfs with label_id, index, domain per row (claim) and dictionary mapping
train_lab_dom = pd.concat([pd.DataFrame(train_labels, columns=['label_id']), train_domains], axis=1)
train_domain_label_map = train_lab_dom.groupby('domain')['label_id'].apply(set).to_dict()

dev_lab_dom = pd.concat([pd.DataFrame(dev_labels, columns=['label_id']), dev_domains], axis=1)
dev_domain_label_map = dev_lab_dom.groupby('domain')['label_id'].apply(set).to_dict()

domain_label_map = train_domain_label_map

In [19]:
domain_label_map

{0: {0, 1, 2, 3, 4, 5, 6, 7, 8},
 1: {0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26},
 2: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},
 3: {0, 1, 2, 3, 4, 5, 6, 7},
 4: {0, 1, 2},
 5: {0, 1, 2, 3, 4, 5},
 6: {0, 1, 2, 3, 4, 5},
 7: {0, 1, 2, 3},
 8: {0, 1, 2, 3, 4, 5},
 9: {0, 1, 2, 3, 4, 5, 6},
 10: {0, 1, 2, 3, 4, 5, 6},
 11: {0, 1},
 12: {0, 1, 2, 3, 4, 5, 6},
 13: {0, 1, 2},
 14: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
 15: {0, 1, 2, 3},
 16: {0, 1, 2, 3, 4, 5},
 17: {0, 1, 2},
 18: {0, 1, 2, 3, 4, 5, 6},
 19: {0, 1, 2, 3, 4},
 20: {0, 1, 2},
 21: {0, 1, 2},
 22: {0, 1},
 23: {0, 1, 2, 3},
 24: {0, 1, 2},
 25: {0, 1, 2}}

# 3. Tokenization & Input Formatting

In this section, we'll transform our dataset into the format that BERT can be trained on.

## 3.1. BERT Tokenizer


To feed our text to BERT, it must be split into tokens, and then these tokens must be mapped to their index in the tokenizer vocabulary.

The tokenization must be performed by the tokenizer included with BERT--the below cell will download this for us. We'll be using the "uncased" version here.


In [20]:
from transformers import RobertaTokenizer

# Load the BERT tokenizer.
print('Loading Roberta tokenizer...')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Loading Roberta tokenizer...


Let's apply the tokenizer to one sentence just to see the output.


When we actually convert all of our sentences, we'll use the `tokenize.encode` function to handle both steps, rather than calling `tokenize` and `convert_tokens_to_ids` separately. 

Before we can do that, though, we need to talk about some of BERT's formatting requirements.

## 3.2. Required Formatting

The above code left out a few required formatting steps that we'll look at here.

*Side Note: The input format to BERT seems "over-specified" to me... We are required to give it a number of pieces of information which seem redundant, or like they could easily be inferred from the data without us explicity providing it. But it is what it is, and I suspect it will make more sense once I have a deeper understanding of the BERT internals.*

We are required to:
1. Add special tokens to the start and end of each sentence.
2. Pad & truncate all sentences to a single constant length.
3. Explicitly differentiate real tokens from padding tokens with the "attention mask".



### Special Tokens



**`[SEP]`**

At the end of every sentence, we need to append the special `[SEP]` token. 

This token is an artifact of two-sentence tasks, where BERT is given two separate sentences and asked to determine something (e.g., can the answer to the question in sentence A be found in sentence B?). 

I am not certain yet why the token is still required when we have only single-sentence input, but it is!


**`[CLS]`**

For classification tasks, we must prepend the special `[CLS]` token to the beginning of every sentence.

This token has special significance. BERT consists of 12 Transformer layers. Each transformer takes in a list of token embeddings, and produces the same number of embeddings on the output (but with the feature values changed, of course!).

![Illustration of CLS token purpose](http://www.mccormickml.com/assets/BERT/CLS_token_500x606.png)

On the output of the final (12th) transformer, *only the first embedding (corresponding to the [CLS] token) is used by the classifier*.

>  "The first token of every sequence is always a special classification token (`[CLS]`). The final hidden state
corresponding to this token is used as the aggregate sequence representation for classification
tasks." (from the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

You might think to try some pooling strategy over the final embeddings, but this isn't necessary. Because BERT is trained to only use this [CLS] token for classification, we know that the model has been motivated to encode everything it needs for the classification step into that single 768-value embedding vector. It's already done the pooling for us!



### Sentence Length & Attention Mask



The sentences in our dataset obviously have varying lengths, so how does BERT handle this?

BERT has two constraints:
1. All sentences must be padded or truncated to a single, fixed length.
2. The maximum sentence length is 512 tokens.

Padding is done with a special `[PAD]` token, which is at index 0 in the BERT vocabulary. The below illustration demonstrates padding out to a "MAX_LEN" of 8 tokens.

<img src="http://www.mccormickml.com/assets/BERT/padding_and_mask.png" width="600">

The "Attention Mask" is simply an array of 1s and 0s indicating which tokens are padding and which aren't (seems kind of redundant, doesn't it?!). This mask tells the "Self-Attention" mechanism in BERT not to incorporate these PAD tokens into its interpretation of the sentence.

The maximum length does impact training and evaluation speed, however. 
For example, with a Tesla K80:

`MAX_LEN = 128  -->  Training epochs take ~5:28 each`

`MAX_LEN = 64   -->  Training epochs take ~2:57 each`







## 3.3. Tokenize Dataset

The transformers library provides a helpful `encode` function which will handle most of the parsing and data prep steps for us.

Before we are ready to encode our text, though, we need to decide on a **maximum sentence length** for padding / truncating to.

The below cell will perform one tokenization pass of the dataset in order to measure the maximum sentence length.

**(A)** an additional for loop will be added for claims that have snippets

NOTE: for claims without snippets, we pass in "tokenizer.unk_token" as the text_pair for argument for the encode_plus method

In [21]:
tokenizer.unk_token

'<unk>'

In [22]:
tokenizer.unk_token == '<unk>'

True

In [23]:
max_len = 0

# For every sentence...
for index, row in train_claims.iterrows():
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    claimID = train_df.iloc[index]['claimID']
    snippets = get_snippets(claimID)
    if(not snippets):
      input_ids = tokenizer.encode(text=row.claim,text_pair = tokenizer.unk_token, add_special_tokens=True)
      max_len = max(max_len, len(input_ids))
    else:
      for snippet in snippets:
        if len(snippet)>0:
            input_ids = tokenizer.encode(text=row.claim,text_pair = snippet, add_special_tokens=True) 
            max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (629 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1310 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (581 > 512). Running this sequence through the model will result in indexing errors


Max sentence length:  1315


Just in case there are some longer test sentences, I'll set the maximum length to 64.


Now we're ready to perform the real tokenization.

The `tokenizer.encode_plus` function combines multiple steps for us:

1. Split the sentence into tokens.
2. Add the special `[CLS]` and `[SEP]` tokens.
3. Map the tokens to their IDs.
4. Pad or truncate all sentences to the same length.
5. Create the attention masks which explicitly differentiate real tokens from `[PAD]` tokens.

The first four features are in `tokenizer.encode`, but I'm using `tokenizer.encode_plus` to get the fifth item (attention masks). Documentation is [here](https://huggingface.co/transformers/main_classes/tokenizer.html?highlight=encode_plus#transformers.PreTrainedTokenizer.encode_plus).


**(A)** modifying to work with snippets, similar to above a second for loop will be added 

"train" boolean param added so we can properly index from original train or val (depending on val of boolean) to get claimID

NOTE: for claims without snippets, we pass in "tokenizer.unk_token" as the text_pair for argument for the encode_plus method

NOTE: unlike working with just claims, the return_token_type_ids param of encode_plus must be set to True, incorporating it will follow the same structure as input_ids and attention_masks (see below)

In [24]:
snippets = [tokenizer.unk_token]
for snippet in snippets:
  print(snippet)

<unk>


In [25]:
from torch.utils.data import TensorDataset
from collections import defaultdict

def encode_claims(df_claims, en_labels, input_seq_length=512,train=True):
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = defaultdict(list)
    attention_masks = defaultdict(list)
    labels = defaultdict(list)
    token_type_ids = defaultdict(list)
    claim_ids = defaultdict(list)

    datasets = {}
    domains_to_tensors = {}

    # For every sentence...
    for index, row in df_claims.iterrows():

        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.

        if(train):
            claimID = train_df.iloc[index]['claimID']
        else:
            claimID = dev_df.iloc[index]['claimID']

        snippets = get_snippets(claimID)

        if(not snippets):
            snippets = [tokenizer.unk_token]
    
        for snippet in snippets:
            if len(snippet)>0:
                encoded_dict = tokenizer.encode_plus(
                            row.claim,
                            text_pair = snippet,             # Sentence to encode.
                            add_special_tokens = True,       # Add '[CLS]' and '[SEP]'
                            max_length = input_seq_length, # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,    # Construct attn. masks.
                            return_tensors = 'pt', 
                            return_token_type_ids = True          # Return pytorch tensors.
                        )

        # Add the encoded sentence to the list.    
                input_ids[row.domain].append(encoded_dict['input_ids'])

            # And its attention mask (simply differentiates padding from non-padding).
                attention_masks[row.domain].append(encoded_dict['attention_mask'])
                
                if en_labels is not None:
                    labels[row.domain].append(en_labels[index])

            # do the same with token_type_ids
                token_type_ids[row.domain].append(encoded_dict['token_type_ids'])
                #labels[row.domain].append(en_labels[index])

                claim_ids[row.domain].append(int(''.join(x for x in claimID if x.isdigit())))

            
    for domain in domain_label_map:
        # Convert the lists into tensors.
        input_temp = torch.cat(input_ids[domain], dim=0)
        attention_masks_temp = torch.cat(attention_masks[domain], dim=0)
        token_type_ids_temp = torch.cat(token_type_ids[domain],dim=0)
        labels_temp = torch.tensor(labels[domain])
        claim_ids_temp = torch.tensor(claim_ids[domain])
        
        # print(input_temp.shape, attention_masks_temp.shape, token_type_ids_temp.shape,labels_temp.shape, claim_ids_temp.shape)
        if labels_temp.size(0)!= 0:
            datasets[domain] = TensorDataset(input_temp, attention_masks_temp,token_type_ids_temp,labels_temp,claim_ids_temp)
        else:
            datasets[domain] = TensorDataset(input_temp, attention_masks_temp,token_type_ids_temp,claim_ids_temp)

    return datasets

In [26]:
train_datasets = encode_claims(train_claims, train_labels,train=True)
dev_datasets = encode_claims(dev_claims, dev_labels,train=False)

## 3.4. Training & Validation Split


We'll also create an iterator for our dataset using the torch DataLoader class. This helps save on memory during training because, unlike a for loop, with an iterator the entire dataset does not need to be loaded into memory.

S-EDITS

In [27]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16
batch_size_val = 1

train_dataloaders = {}
validation_dataloaders = {}

for idx, domain in enumerate(domain_label_map):

    # Create the DataLoaders for our training and validation sets.
    # We'll take training samples in random order. 
    train_dataloaders[domain] = DataLoader(
                train_datasets[domain],  # The training samples.
                sampler = RandomSampler(train_datasets[domain]), # Select batches randomly
                batch_size = batch_size # Trains with this batch size.
            )

    # For validation the order doesn't matter, so we'll just read them sequentially.
    validation_dataloaders[domain] = DataLoader(
                dev_datasets[domain], # The validation samples.
                sampler = SequentialSampler( dev_datasets[domain]), # Pull out batches sequentially.
                batch_size = batch_size_val # Evaluate with this batch size.
            )

# 4. Train Our Classification Model

Now that our input data is properly formatted, it's time to fine tune the BERT model. 

## 4.1. BertForSequenceClassification

For this task, we first want to modify the pre-trained BERT model to give outputs for classification, and then we want to continue training the model on our dataset until that the entire model, end-to-end, is well-suited for our task. 

Thankfully, the huggingface pytorch implementation includes a set of interfaces designed for a variety of NLP tasks. Though these interfaces are all built on top of a trained BERT model, each has different top layers and output types designed to accomodate their specific NLP task.  

Here is the current list of classes provided for fine-tuning:
* BertModel
* BertForPreTraining
* BertForMaskedLM
* BertForNextSentencePrediction
* **BertForSequenceClassification** - The one we'll use.
* BertForTokenClassification
* BertForQuestionAnswering

The documentation for these can be found under [here](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html).



We'll be using [BertForSequenceClassification](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#bertforsequenceclassification). This is the normal BERT model with an added single linear layer on top for classification that we will use as a sentence classifier. As we feed input data, the entire pre-trained BERT model and the additional untrained classification layer is trained on our specific task. 


OK, let's load BERT! There are a few different pre-trained BERT models available. "bert-base-uncased" means the version that has only lowercase letters ("uncased") and is the smaller version of the two ("base" vs "large").

The documentation for `from_pretrained` can be found [here](https://huggingface.co/transformers/v2.2.0/main_classes/model.html#transformers.PreTrainedModel.from_pretrained), with the additional parameters defined [here](https://huggingface.co/transformers/v2.2.0/main_classes/configuration.html#transformers.PretrainedConfig).

In [28]:
from torch import nn

class MultiLinear(nn.Module):
    def __init__(self, hidden_size, domain_label_map):
        super(MultiLinear, self).__init__()
        self.linear_models = {}
        for domain in domain_label_map:
            self.linear_models[domain] = nn.Linear(hidden_size, len(domain_label_map[domain])).to(device)

    def forward(self, domain, input):
        return self.linear_models[domain].forward(input)

In [29]:
from transformers import RobertaForSequenceClassification, RobertaModel
from torch.nn import CrossEntropyLoss

class RobertaForSequenceClassificationMultiDomain(RobertaForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.roberta = RobertaModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = MultiLinear(config.hidden_size, domain_label_map)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, \
        inputs_embeds=None, \
        m_labels=None,
        m_domain=None):
        """
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`, defaults to :obj:`None`):
            Labels for computing the sequence classification/regression loss.
            Indices should be in :obj:`[0, ..., config.num_labels - 1]`.
            If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).

    Returns:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.BertConfig`) and inputs:
        loss (:obj:`torch.FloatTensor` of shape :obj:`(1,)`, `optional`, returned when :obj:`label` is provided):
            Classification (or regression if config.num_labels==1) loss.
        logits (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, config.num_labels)`):
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        hidden_states (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_hidden_states=True``):
            Tuple of :obj:`torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer)
            of shape :obj:`(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_attentions=True``):
            Tuple of :obj:`torch.FloatTensor` (one for each layer) of shape
            :obj:`(batch_size, num_heads, sequence_length, sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.

    Examples::

        from transformers import BertTokenizer, BertForSequenceClassification
        import torch

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)

        loss, logits = outputs[:2]

        """

        outputs = self.roberta(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask,
                            inputs_embeds=inputs_embeds)

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(domain, pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here
        
        if self.training:

            loss_fct = CrossEntropyLoss()

            loss = loss_fct(logits.view(-1, len(domain_label_map[m_domain.item()])), m_labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [30]:
from transformers import RobertaForSequenceClassification, AdamW, RobertaConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = RobertaForSequenceClassificationMultiDomain.from_pretrained(
    "roberta-base", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(le.classes_), # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda();

Just for curiosity's sake, we can browse all of the model's parameters by name here.

In the below cell, I've printed out the names and dimensions of the weights for:

1. The embedding layer.
2. The first of the twelve transformers.
3. The output layer.




In [31]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The Roberta model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The Roberta model has 199 different named parameters.

==== Embedding Layer ====

roberta.embeddings.word_embeddings.weight               (50265, 768)
roberta.embeddings.position_embeddings.weight             (514, 768)
roberta.embeddings.token_type_embeddings.weight             (1, 768)
roberta.embeddings.LayerNorm.weight                           (768,)
roberta.embeddings.LayerNorm.bias                             (768,)

==== First Transformer ====

roberta.encoder.layer.0.attention.self.query.weight       (768, 768)
roberta.encoder.layer.0.attention.self.query.bias             (768,)
roberta.encoder.layer.0.attention.self.key.weight         (768, 768)
roberta.encoder.layer.0.attention.self.key.bias               (768,)
roberta.encoder.layer.0.attention.self.value.weight       (768, 768)
roberta.encoder.layer.0.attention.self.value.bias             (768,)
roberta.encoder.layer.0.attention.output.dense.weight     (768, 768)
roberta.encoder.layer.0.attention.output.dense.bias         

## 4.2. Optimizer & Learning Rate Scheduler

In [32]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [33]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 1

total_steps = 0
for domain in domain_label_map:
    # Total number of training steps is [number of batches] x [number of epochs]. 
    # (Note that this is not the same as the number of training samples).
    total_steps += len(train_dataloaders[domain]) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

## 4.3. Training Loop

Below is our training loop. There's a lot going on, but fundamentally for each pass in our loop we have a trianing phase and a validation phase. 

> *Thank you to [Stas Bekman](https://ca.linkedin.com/in/stasbekman) for contributing the insights and code for using validation loss to detect over-fitting!*

**Training:**
- Unpack our data inputs and labels
- Load data onto the GPU for acceleration
- Clear out the gradients calculated in the previous pass. 
    - In pytorch the gradients accumulate by default (useful for things like RNNs) unless you explicitly clear them out.
- Forward pass (feed input data through the network)
- Backward pass (backpropagation)
- Tell the network to update parameters with optimizer.step()
- Track variables for monitoring progress

**Evalution:**
- Unpack our data inputs and labels
- Load data onto the GPU for acceleration
- Forward pass (feed input data through the network)
- Compute loss on our validation data and track variables for monitoring progress

Pytorch hides all of the detailed calculations from us, but we've commented the code to point out which of the above steps are happening on each line. 

> *PyTorch also has some [beginner tutorials](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py) which you may also find helpful.*

Define a helper function for calculating accuracy.

In [34]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [35]:
from sklearn.metrics import f1_score

def f1_macro(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat, average='macro')

def f1_micro(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat, average='micro')

Helper function for formatting elapsed times as `hh:mm:ss`


In [36]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


We're ready to kick off the training!

In [0]:
# SR edits - trying to run the validation separately first 

# import random
# from scipy import stats
# from collections import defaultdict

# scores = {k:defaultdict(list) for k in domain_label_map.keys()} #scores is a dictionary with key as domaid_id and the value as a defaultdict with claims as the keys and predictions list as the values
# labels = {k:defaultdict(int) for k in domain_label_map.keys()} #labels is a doctionary with key as domain_id and the values are the true labels

# domains = list(domain_label_map.keys())
# random.shuffle(domains)

# dataloaders_iters = {}
# dataloader_batch_tracker = {}

# for domain in validation_dataloaders:
#     dataloaders_iters[domain] = iter(validation_dataloaders[domain])
#     dataloader_batch_tracker[domain] = 0
# step = 0
# validation_domains_list = domains.copy()

# # scores=[]
# # classes_val=[]
# # current_input=torch.zeros(3, device=device)

# while validation_domains_list:
#     step += 1
#     domain = random.choice(validation_domains_list)
    
#     try:
#         batch = next(dataloaders_iters[domain])
#     except StopIteration: 
#         validation_domains_list.remove(domain)
#         continue

#     # Unpack this training batch from our dataloader. 
#     #
#     # As we unpack the batch, we'll also copy each tensor to the GPU using 
#     # the `to` method.
#     #
#     # `batch` contains three pytorch tensors:
#     #   [0]: input ids 
#     #   [1]: attention masks
#     #   [2]: token type ids 
#     #   [3]: labels 
#     b_input_ids = batch[0].to(device)
    
#     # print(b_input_ids[:10])
#     # print(b_input_ids[:10].shape)
#     b_input_mask = batch[1].to(device)
#     # print(b_input_ids)
#     b_token_type_ids = batch[2].to(device)
#     # print(b_input_ids)
#     b_labels = batch[3].to(device)
#     # print(b_input_ids)
#     b_domain = torch.tensor([domain]).to(device)

#     with torch.no_grad():        

#         # Forward pass, calculate logit predictions.
#         # token_type_ids is the same as the "segment ids", which 
#         # differentiates sentence 1 and 2 in 2-sentence tasks.
#         # The documentation for this `model` function is here: 
#         # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
#         # Get the "logits" output by the model. The "logits" are the output
#         # values prior to applying an activation function like the softmax.
#         (loss, logits) = model(b_input_ids, 
#                             token_type_ids=b_token_type_ids, 
#                             attention_mask=b_input_mask,
#                             m_labels=b_labels,
#                             m_domain=b_domain)
#     s = str(b_input_ids[0,:5].cpu().tolist()) #converting list to an immutable object so that it can be used as the keys of a hash table
#     scores[b_domain.item()][s].append(torch.argmax(logits[0]).item()) 
#     labels[b_domain.item()][s] = b_labels[0].item()
  

In [37]:
from sklearn.metrics import precision_recall_fscore_support

In [38]:
def f_score_calculation(scores,labels):
    '''
    scores: dict of domains as keys and values as another dictionary with claims as keys and predicted classes as values
    labels: dict of domains as keys and values as another dictionary with claims as keys and actual classes as values

    Returns:
    f1_micro: dictionary of macro f1 scores for each domain
    f1_micro: dictionary of micro f1 scores for each domain
    '''
    f1_macro = {}
    f1_micro = {}
    
    for dom in labels.keys():
        y_pred = [stats.mode(scores[dom][k])[0][0] for k in scores[dom].keys()]
        y_true = [labels[dom][k] for k in scores[dom].keys()]
        f1_macro[dom] = precision_recall_fscore_support(y_true,y_pred,average='macro')[2]
        f1_micro[dom] = precision_recall_fscore_support(y_true,y_pred,average='micro')[2]
    return f1_micro, f1_macro

In [39]:
# macro,micro = f_score_calculation(scores,labels)

In [40]:
# macro,micro

In [41]:
# scores[2]

In [42]:
# len(scores.keys())

In [43]:
# str(b_input_ids[0,:5].cpu().tolist())


In [44]:
# torch.sum(current_input == b_input_ids[:,:5])==5


In [45]:
# logits[0]

In [46]:
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [0]:
# torch.save(model.state_dict(),'model.pth')

In [0]:
import random
import numpy as np
import random
from collections import defaultdict
from scipy import stats

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
best_val = -10000

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    domains = list(domain_label_map.keys())
    random.shuffle(domains)

    dataloaders_iters = {}
    dataloader_batch_tracker = {}
    for domain in train_dataloaders:
        dataloaders_iters[domain] = iter(train_dataloaders[domain])
        dataloader_batch_tracker[domain] = 0
    train_domains_list = domains.copy()
    total_train_loss = 0
    step = 0

    while train_domains_list:
        domain = random.choice(train_domains_list)
        dataloader_batch_tracker[domain] += 1
        try:
            batch = next(dataloaders_iters[domain])
        except StopIteration: 
            train_domains_list.remove(domain)
            continue

        step += 1
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(dataloader_batch_tracker[domain], len(train_dataloaders[domain]), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains four pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: token type ids
        #   [3]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_token_type_ids = batch[2].to(device)
        b_labels = batch[3].to(device)

        b_domain = torch.tensor([domain]).to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                            token_type_ids=b_token_type_ids, 
                            attention_mask=b_input_mask, 
                            m_labels=b_labels,
                            m_domain=b_domain)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / step         

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

# ========================================
#               Validation
# ========================================
# After the completion of each training epoch, measure our performance on
# our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    scores = {k:defaultdict(list) for k in domain_label_map.keys()} #scores is a dictionary with key as domaid_id and the value as a defaultdict with claims as the keys and predictions list as the values
    labels = {k:defaultdict(int) for k in domain_label_map.keys()} #labels is a doctionary with key as domain_id and the values are the true labels


    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # F1 Scores
    f1_macro_score = 0
    f1_micro_score = 0

    for domain in validation_dataloaders:
        dataloaders_iters[domain] = iter(validation_dataloaders[domain])
        dataloader_batch_tracker[domain] = 0
    step = 0
    validation_domains_list = domains.copy()

    while validation_domains_list:
        step += 1
        domain = random.choice(validation_domains_list)

        try:
            batch = next(dataloaders_iters[domain])
        except StopIteration: 
            validation_domains_list.remove(domain)
            continue

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: token type ids 
        #   [3]: labels 
        b_input_ids = batch[0].to(device)
        #print(b_input_ids)
        b_input_mask = batch[1].to(device)
        b_token_type_ids = batch[2].to(device)
        b_labels = batch[3].to(device)
        b_domain = torch.tensor([domain]).to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                token_type_ids=b_token_type_ids, 
                                attention_mask=b_input_mask,
                                m_labels=b_labels,
                                m_domain=b_domain)


        # SR 
        s = str(b_input_ids[0,:5].cpu().tolist()) #converting list to an immutable object so that it can be used as the keys of a hash table
        scores[b_domain.item()][s].append(torch.argmax(logits[0]).item()) 
        labels[b_domain.item()][s] = b_labels[0].item()

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        f1_macro_score += f1_macro(logits, label_ids) 
        f1_micro_score += f1_micro(logits, label_ids) 

    # Accuracy for vote on validation method
    #print("Accuracy for VOTE ON VALIDATION method = {0:.2f}".format(np.mean([stats.mode[v] for k,v in score.items()])))
    micro_f1_vote,macro_f1_vote = f_score_calculation(scores,labels)
    avg_macro_f1_vote = np.mean(list(macro_f1_vote.values()))
    avg_micro_f1_vote = np.mean(list(micro_f1_vote.values()))
    
    if avg_micro_f1_vote>best_val:
        torch.save(model.state_dict(),'model.pth')
        best_val = avg_micro_f1_vote
    
    print("F1 Macro VOTE: {0:.2f}".format(avg_macro_f1_vote))
    print("F1 Micro VOTE: {0:.2f}".format(avg_micro_f1_vote))
    print('By domain, micro VOTE')
    print(micro_f1_vote)
    print('By domain, macro VOTE')
    print(macro_f1_vote)
    
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / step
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Report the final f1 macro score for this validation run.
    avg_val_f1_macro = f1_macro_score / step
    print("  F1 Macro: {0:.2f}".format(avg_val_f1_macro))

    # Report the final f1 micro score for this validation run.
    avg_val_f1_micro = f1_micro_score / step
    print("  F1 Micro: {0:.2f}".format(avg_val_f1_micro))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / step

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 1 ========
Training...


RuntimeError: ignored

In [0]:
avg_macro_f1_vote = np.mean(list(macro_f1_vote.values()))

All data, separate classifier heads, 128 input length, batch size 16

Running Validation...
  Accuracy: 0.45
  F1 Macro: 0.24
  F1 Micro: 0.45
  Validation Loss: 1.72
  Validation took: 0:01:00

All data, separate classifier heads, 128 input length, batch size 16. random batches. 0.27 F1 Macro. 0.42 F1 macro.

All data, separate classifier heads, 512 input length, batch size 16. random batches. 0.26 F1 Macro. 0.42 F1 macro.

Let's view the summary of the training process.

In [0]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

Notice that, while the the training loss is going down with each epoch, the validation loss is increasing! This suggests that we are training our model too long, and it's over-fitting on the training data. 

(For reference, we are using 7,695 training samples and 856 validation samples).

Validation Loss is a more precise measure than accuracy, because with accuracy we don't care about the exact output value, but just which side of a threshold it falls on. 

If we are predicting the correct answer, but with less confidence, then validation loss will catch this, while accuracy will not.

In [0]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

# 5. Performance On Test Set

Now we'll load the holdout dataset and prepare inputs just as we did with the training set. Then we'll evaluate predictions using [Matthew's correlation coefficient](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html) because this is the metric used by the wider NLP community to evaluate performance on CoLA. With this metric, +1 is the best score, and -1 is the worst score. This way, we can see how well we perform against the state of the art models for this specific task.

### 5.1. Data Preparation



We'll need to apply all of the same steps that we did for the training data to prepare our test data set.

In [0]:
# def encode_claims_test(df_claims, en_labels = None, input_seq_length=128):
#     # Tokenize all of the sentences and map the tokens to thier word IDs.
#     input_ids = defaultdict(list)
#     attention_masks = defaultdict(list)
#     labels = defaultdict(list)

#     datasets = {}
#     domains_to_tensors = {}

#     # For every sentence...
#     for index, row in df_claims.iterrows():

#         # `encode_plus` will:
#         #   (1) Tokenize the sentence.
#         #   (2) Prepend the `[CLS]` token to the start.
#         #   (3) Append the `[SEP]` token to the end.
#         #   (4) Map tokens to their IDs.
#         #   (5) Pad or truncate the sentence to `max_length`
#         #   (6) Create attention masks for [PAD] tokens.
#         encoded_dict = tokenizer.encode_plus(
#                             row.claim,                            # Sentence to encode.
#                             add_special_tokens = True,       # Add '[CLS]' and '[SEP]'
#                             max_length = input_seq_length, # Pad & truncate all sentences.
#                             pad_to_max_length = True,
#                             return_attention_mask = True,    # Construct attn. masks.
#                             return_tensors = 'pt',           # Return pytorch tensors.
#                     )

#         # Add the encoded sentence to the list.    
#         input_ids[row.domain].append(encoded_dict['input_ids'])
        
#         # And its attention mask (simply differentiates padding from non-padding).
#         attention_masks[row.domain].append(encoded_dict['attention_mask'])

#         if en_labels is not None:
#           labels[row.domain].append(en_labels[index])

#     for domain in domain_label_map:
#         # Convert the lists into tensors.
#         input_temp = torch.cat(input_ids[domain], dim=0)
#         attention_masks_temp = torch.cat(attention_masks[domain], dim=0)
#         if en_labels is not None:
#           labels_temp = torch.tensor(labels[domain])
        
#           datasets[domain] = TensorDataset(input_temp, attention_masks_temp, labels_temp)
#         else:
#           datasets[domain] = TensorDataset(input_temp, attention_masks_temp)

#     return datasets

In [49]:
test_df = pd.read_csv("multi-fc/test.tsv", delimiter='\t', header=None, quoting=csv.QUOTE_NONE, \
                 names= ['claimID', 'claim', 'claimURL', 'reason', 'categories', 'speaker', \
                  'checker', 'tags', 'articleTitle', 'publishDate', 'claimDate', 'entities'])

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(test_df.shape[0]))

index_to_add = test_df.index[test_df.isnull()['claim'] == True].tolist()
test_df = drop_malformed(test_df)

##gives df of domain for every claim
test_domains = generate_domain_df(test_df)
## replaces domain name column with domain id
test_domains = test_domains.replace({'domain': domain_to_domain_id})

test_claims = pd.concat([test_df[['claim','claimID']], test_domains], axis=1)

test_datasets = encode_claims(test_claims, en_labels=None, train=False)

#Claim idx domain map used to correctly sort output
test_domain_idx_map = test_domains.groupby('domain')['index'].apply(list).to_dict()

Number of test sentences: 3,491



In [ ]:
test_datasets

In [50]:
test_dataloaders = {}

for idx, domain in enumerate(domain_label_map):

    # Create the DataLoaders for our test set.
    # For test the order doesn't matter, so we'll just read them sequentially.
    
    test_dataloaders[domain] = DataLoader(
                test_datasets[domain], # The validation samples.
                sampler = SequentialSampler( test_datasets[domain]), # Pull out batches sequentially.
                batch_size = 1 # Evaluate with this batch size.
            )

## 5.2. Evaluate on Test Set



With the test set prepared, we can apply our fine-tuned model to generate predictions on the test set.

In [51]:
print("Running Testing...")
import random
t0 = time.time()
predictions = []
pred_domains = []

scores = {k:defaultdict(list) for k in domain_label_map.keys()} #scores is a dictionary with key as domaid_id and the value as a defaultdict with claims as the keys and predictions list as the values
#labels = {k:defaultdict(int) for k in domain_label_map.keys()} #labels is a doctionary with key as domain_id and the values are the true labels
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
model.eval()
dataloaders_iters = {}
dataloader_batch_tracker = {}
domains = list(domain_label_map.keys())
for domain in test_dataloaders:
    dataloaders_iters[domain] = iter(test_dataloaders[domain])
    dataloader_batch_tracker[domain] = 0
step = 0
test_domains_list = domains.copy()

for domain in test_dataloaders:
    for step, batch in enumerate(test_dataloaders[domain]):
        
        pred_domains.append(domain)

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using 
            # the `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_token_type_ids = batch[2].to(device)
        b_domain = torch.tensor([domain]).to(device)
        claim_id = batch[3].to(device)
            
            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
            logits = model(b_input_ids, 
                                    token_type_ids=b_token_type_ids, 
                                    attention_mask=b_input_mask,
                                    m_domain=b_domain)
        
        #s = str(b_input_ids[0,:5].cpu().tolist()) #converting list to an immutable object so that it can be used as the keys of a hash table
        
        # SR - claim_id.item instead of claim_id (because claim_id is also a tensor)
        scores[b_domain.item()][claim_id.item()].append(torch.argmax(logits[0]).item()) 
        #labels[b_domain.item()][s] = b_labels[0].item()
            
#         # Accumulate the validation loss.
#     logits = outputs[1]
#   # Move logits and labels to CPU
#     logits = logits.detach().cpu().numpy()
  
#   # Store predictions and true labels
#     predictions.append(logits)




Running Testing...


In [88]:
#claim_id.item()

10525

In [59]:
scores[0].keys()

dict_keys([244, 534, 1936, 2608, 13769, 76, 2423, 6530, 12571, 275, 664, 12597, 5743, 834, 2167, 301, 12207, 2305, 1995, 14583, 1800, 2113, 2021, 3466, 12366, 504, 11472, 1901, 305, 884, 2939, 2216, 3846, 1143, 3132, 2636, 12261, 624, 25, 1921, 8485, 2046, 3878, 81, 143, 4173, 1347, 1628, 2332, 490, 700, 1364, 4998, 871, 5527, 5971, 4290, 7928, 1833, 7226, 5600, 1391, 2051, 3320, 15294, 12108, 294, 4332, 364, 10598, 1267, 4034, 1351, 2981, 2642, 10960, 23, 73, 717, 2229, 12125, 157, 758, 259, 3509, 6541, 14765, 134, 7657, 3599, 9944, 6178, 11528, 375, 346, 180, 10945, 2005, 8939, 2023, 1187, 209, 13224, 3891, 131, 16, 4497, 242, 773, 1100, 2982, 2767, 12172, 4368, 13721, 1006, 406, 3749, 13154, 1224, 3971, 7648, 151, 630, 383, 8198, 2298, 56, 9157, 175, 1079, 2206, 388, 766, 1308, 587, 3240, 582, 5751, 911, 5479, 313, 3198, 11602, 11422, 4892, 992, 671, 426, 8713, 10016, 3453, 12045, 3482, 7770, 109, 11149, 4876, 4843, 7032, 9511, 623, 5368, 96, 8886, 2901, 49, 1043, 4575, 2022, 4634, 

In [60]:
import copy
from scipy import stats
from statistics import mode

# pred_dict = copy.deepcopy(scores)
pred_dict = {k: defaultdict(int) for k in domain_label_map.keys()}

# y_pred = [stats.mode(scores[dom][k])[0][0] for k in scores[dom].keys()]

for domain in test_dataloaders:
    # for claim_id in test_dataloaders[domain]:
    for batch in test_dataloaders[domain]:
        # print(domain)
        # print(stats.mode(scores[domain.values()][claim_id]))
        claim_id = batch[3].item()
        
        # pred_dict[domain][claim_id] = stats.mode(scores[domain][claim_id])
        pred_dict[domain][claim_id] = stats.mode(scores[domain][claim_id])[0][0]
        # print(pred_dict[domain][claim_id])

In [62]:
train_labels_map_to_domain

{0: {'half flip': 0,
  'mostly false': 1,
  'true': 2,
  'half-true': 3,
  'no flip': 4,
  'pants on fire!': 5,
  'mostly true': 6,
  'full flop': 7,
  'false': 8},
 1: {'scam!': 0,
  'investigation pending!': 1,
  'authorship confirmed!': 2,
  'commentary!': 3,
  'virus!': 4,
  'truth! & fiction!': 5,
  'unproven!': 6,
  'inaccurate attribution!': 7,
  'incorrect attribution!': 8,
  'fiction! & satire!': 9,
  'correct attribution!': 10,
  'truth! & misleading!': 11,
  'fiction!': 12,
  'none': 13,
  'outdated!': 14,
  'truth!': 15,
  'disputed!': 16,
  'opinion!': 17,
  'misleading!': 18,
  'truth! & outdated!': 19,
  'truth! & disputed!': 20,
  'mostly truth!': 21,
  'confirmed authorship!': 22,
  'truth! & unproven!': 23,
  'grass roots movement!': 24,
  'previously truth! now resolved!': 25,
  'mostly fiction!': 26},
 2: {'mixture': 0,
  'scam': 1,
  'mostly false': 2,
  'true': 3,
  'unproven': 4,
  'misattributed': 5,
  'miscaptioned': 6,
  'legend': 7,
  'mostly true': 8,
  'fal

In [63]:
domain

0

In [64]:
claim_id

14759

In [67]:
preds = []
for index, row in test_claims.iterrows():
    
    domain = row.domain
    claim_id = int(''.join(x for x in row.claimID if x.isdigit()))
    p = list(train_labels_map_to_domain[domain].keys())[pred_dict[domain][claim_id]]
    preds.append(p)

In [68]:
preds.insert(index_to_add[0],'false')

In [69]:
pred_dict

{0: defaultdict(int,
             {244: 1,
              534: 5,
              1936: 4,
              2608: 0,
              13769: 0,
              76: 0,
              2423: 0,
              6530: 0,
              12571: 1,
              275: 1,
              664: 1,
              12597: 0,
              5743: 0,
              834: 0,
              2167: 0,
              301: 0,
              12207: 0,
              2305: 0,
              1995: 4,
              14583: 0,
              1800: 0,
              2113: 0,
              2021: 5,
              3466: 5,
              12366: 0,
              504: 0,
              11472: 5,
              1901: 1,
              305: 0,
              884: 0,
              2939: 0,
              2216: 0,
              3846: 1,
              1143: 0,
              3132: 0,
              2636: 1,
              12261: 0,
              624: 1,
              25: 0,
              1921: 0,
              8485: 1,
              2046: 1,
              3878:

In [70]:
len(preds)

3491

In [72]:
preds

['half flip',
 'half flip',
 'false',
 'half flip',
 'mixture',
 'half flip',
 'mixture',
 'mixture',
 'mixture',
 'mixture',
 '0',
 '2',
 'promise kept',
 'scam!',
 'scam!',
 'half flip',
 '0',
 'scam!',
 'half flip',
 'mostly false',
 'half flip',
 'mostly false',
 'half flip',
 'mixture',
 'half flip',
 'half flip',
 'half flip',
 'mixture',
 'half flip',
 'half flip',
 'half flip',
 'half flip',
 'promise kept',
 '0',
 'miscaptioned',
 '0',
 'half flip',
 'half flip',
 'mixture',
 'cherry picks',
 'half flip',
 'half flip',
 'half flip',
 'half flip',
 'half flip',
 'half flip',
 'misleading',
 'half flip',
 'half flip',
 'we rate this claim false',
 'half flip',
 'half flip',
 'half flip',
 'mixture',
 'scam!',
 'mixture',
 'half flip',
 'scam!',
 'half flip',
 'mixture',
 'half flip',
 'determination: a stretch',
 '0',
 'half flip',
 '0',
 'mixture',
 'half flip',
 'half flip',
 '0',
 'half flip',
 'factscan score: misleading',
 'half flip',
 'promise kept',
 'scam!',
 'half flip

In [ ]:
# train_labels = pd.concat([train_df['label'], train_domains], axis=1)
# train_labels_map_to_domain = train_labels.groupby('domain')['label'].apply(set).apply(list).to_dict()
# print(train_labels_map_to_domain)

In [73]:
# write preds to a file

with open('test.predict', 'w') as filehandle:
    filehandle.writelines("%s\n" % pred for pred in preds)

In [0]:
# import itertools
# y_pred = []
# batch_sizes = []
# for i in range(len(predictions)):
  
#   # The predictions for this batch are a 5-column ndarray with a row for each batch (32). 
#   #Pick the label with the highest value and consider the index = class
#   pred_labels_i = list(np.argmax(predictions[i], axis=1))
#   current_domain = train_labels_map_to_domain[pred_domains[i]]

#   batch_sizes.append(len(pred_labels_i))
#   for j in pred_labels_i:
#     label = list(current_domain.keys())[list(current_domain.values()).index(j)]
#     y_pred.append(label)

# #Get domain respective to each prediction
# domain_order = []
# for idx, domain in enumerate(pred_domains):
#   batch_size = batch_sizes[idx]
#   for i in range(batch_size):
#     domain_order.append(domain)

# #Obtain claim index of each prediction
# new_index = []
# for domain in domain_order:
#   new_index.append(test_domain_idx_map[domain][0])
#   test_domain_idx_map[domain].pop(0)

# #Sort predictions based on original index 
# y_pred = [x for _,x in sorted(zip(new_index,y_pred))]
# y_pred[0:5]

Accuracy on the CoLA benchmark is measured using the "[Matthews correlation coefficient](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html)" (MCC).

We use MCC here because the classes are imbalanced:


Now we'll combine the results for all of the batches and calculate our final MCC score.

# Conclusion

This post demonstrates that with a pre-trained BERT model you can quickly and effectively create a high quality model with minimal effort and training time using the pytorch interface, regardless of the specific NLP task you are interested in.

# Appendix


## A1. Saving & Loading Fine-Tuned Model

This first cell (taken from `run_glue.py` [here](https://github.com/huggingface/transformers/blob/35ff345fc9df9e777b27903f11fa213e4052595b/examples/run_glue.py#L495)) writes the model and tokenizer out to disk.

In [0]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


Let's check out the file sizes, out of curiosity.

In [0]:
!ls -l --block-size=K ./model_save/

The largest file is the model weights, at around 418 megabytes.

In [0]:
!ls -l --block-size=M ./model_save/pytorch_model.bin

To save your model across Colab Notebook sessions, download it to your local machine, or ideally copy it to your Google Drive.

In [0]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
    drive.mount('/content/drive')

In [0]:
# Copy the model files to a directory in your Google Drive.
!cp -r ./model_save/ "./drive/Shared drives/ChrisMcCormick.AI/Blog Posts/BERT Fine-Tuning/"

The following functions will load the model back from disk.

In [0]:
# Load a trained model and vocabulary that you have fine-tuned
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

## A.2. Weight Decay



The huggingface example includes the following code block for enabling weight decay, but the default decay rate is "0.0", so I moved this to the appendix.

This block essentially tells the optimizer to not apply weight decay to the bias terms (e.g., $ b $ in the equation $ y = Wx + b $ ). Weight decay is a form of regularization--after calculating the gradients, we multiply them by, e.g., 0.99.

In [0]:
# This code is taken from:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L102

# Don't apply weight decay to any parameters whose names include these tokens.
# (Here, the BERT doesn't have `gamma` or `beta` parameters, only `bias` terms)
no_decay = ['bias', 'LayerNorm.weight']

# Separate the `weight` parameters from the `bias` parameters. 
# - For the `weight` parameters, this specifies a 'weight_decay_rate' of 0.01. 
# - For the `bias` parameters, the 'weight_decay_rate' is 0.0. 
optimizer_grouped_parameters = [
    # Filter for all parameters which *don't* include 'bias', 'gamma', 'beta'.
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.1},
    
    # Filter for parameters which *do* include those.
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

# Note - `optimizer_grouped_parameters` only includes the parameter values, not 
# the names.

# Revision History

**Version 3** - *Mar 18th, 2020* - (current)
* Simplified the tokenization and input formatting (for both training and test) by leveraging the `tokenizer.encode_plus` function. 
`encode_plus` handles padding *and* creates the attention masks for us.
* Improved explanation of attention masks.
* Switched to using `torch.utils.data.random_split` for creating the training-validation split.
* Added a summary table of the training statistics (validation loss, time per epoch, etc.).
* Added validation loss to the learning curve plot, so we can see if we're overfitting. 
    * Thank you to [Stas Bekman](https://ca.linkedin.com/in/stasbekman) for contributing this!
* Displayed the per-batch MCC as a bar plot.

**Version 2** - *Dec 20th, 2019* - [link](https://colab.research.google.com/drive/1Y4o3jh3ZH70tl6mCd76vz_IxX23biCPP)
* huggingface renamed their library to `transformers`. 
* Updated the notebook to use the `transformers` library.

**Version 1** - *July 22nd, 2019*
* Initial version.

## Further Work

* It might make more sense to use the MCC score for “validation accuracy”, but I’ve left it out so as not to have to explain it earlier in the Notebook.
* Seeding -- I’m not convinced that setting the seed values at the beginning of the training loop is actually creating reproducible results…
* The MCC score seems to vary substantially across different runs. It would be interesting to run this example a number of times and show the variance.
